In [1]:
import json
import torch
import torch.nn as nn
import random
from tqdm import tqdm
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
supporting_texts = json.load(open('../data/supporting.json'))
refuting_texts = json.load(open('../data/refuting.json'))

In [3]:
print(supporting_texts[10])

Evidence:
He composed the scores to the television series Lost, Alias and Fringe, the video ' game series Medal of Honor and Call of Duty and many films such as The Incredibles, Ratatouille, Up, Mission: Impossible - Ghost Protocol, Dawn of the Planet of the Apes, Jurassic World, Inside Out, Star Trek Beyond, Doctor Strange, Rogue One and Spider-Man: Homecoming.

Claim:
Michael Giacchino composed the score for Doctor Strange.


In [4]:
import random
split = 0.8

_prompt = '\n\nThe evidence supports the claim:\n'
all_list = [item + _prompt + 'Y' for item in supporting_texts]
all_list += [item + _prompt + 'N' for item in refuting_texts]
random.shuffle(all_list)
train_list = all_list[:int(len(all_list) * split)]
dev_list = all_list[int(len(all_list) * split):]

In [5]:
del all_list

In [6]:
#json.dump(train_list, open('../data/train_list.json', 'w'))
#json.dump(dev_list, open('../data/dev_list.json', 'w'))

In [17]:
train_list = json.load(open('../data/train_list.json'))
dev_list = json.load(open('../data/dev_list.json'))

In [18]:
print(train_list[0])

Evidence:
Shot in a mockumentary format, it follows the contestants in a beauty pageant called the Sarah Rose Cosmetics Mount Rose American Teen Princess Pageant, held in the small fictional town of Mount Rose, Minnesota, in which various contestants die in suspicious ways.

Claim:
Drop Dead Gorgeous follows the contestants in the Sarah Rose Cosmetics Mount Rose American Child Princess Pageant.

The evidence supports the claim:
N


In [19]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def batchify(data, n):
    len_dict = {}
    for item in data:
        length = item.shape[1]
        try:
            len_dict[length].append(item)
        except:
            len_dict[length] = [item]

    batch_chunks = []
    for k in len_dict.keys():
        vectors = len_dict[k]
        batch_chunks += chunks(vectors, n)

    batches = []
    for chunk in batch_chunks:
        inputs = torch.stack([item[0] for item in chunk])
        batches.append((inputs))

    return batches

In [20]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [ ]:
_limit = 1024
data = []
total_skipped = 0
for item in train_list:
    tokens = tokenizer.encode(item, return_tensors='pt')
    if tokens.shape[1] > _limit:
        total_skipped += 1
        continue
    data.append(tokens)
print(f'Skipped {total_skipped} out of {len(train_list)}')

In [ ]:
train_batches = batchify(data, 2)

In [ ]:
def train(train_model, batches, optimizer, criterion):
    total_loss = 0.
    for i, batch in tqdm(enumerate(batches), total=len(batches)):
        model.train()
        inputs = batch
        optimizer.zero_grad()
        loss = train_model(inputs.cuda(), labels=inputs.cuda())[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(train_model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(batches)

In [ ]:
from torch.optim.lr_scheduler import StepLR

random.shuffle(train_batches)
scheduler = StepLR(optimizer, step_size=2, gamma=0.8)
for epoch in range(10):
    random.shuffle(train_batches)
    loss = train(model, train_batches, optimizer, criterion)z
    #test(model, dev_list[:2000])
    print('Epoch:', epoch, 'Loss:', loss)
    torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict()},
                'save_fever' + str(epoch))
    scheduler.step()

In [96]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.cuda()
checkpoint = torch.load('save_fever2')
model.load_state_dict(checkpoint['model_state_dict'])
_ = model.eval()

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import sys
import traceback

def test(model, data):
    model.eval()
    tp = 0
    fp = 0
    fn = 0

    skipped = 0

    for item in tqdm(data):
        expected = get_answer_from_text(item)
        predicted = ''
        try:
            predicted = generate_answer(model, item)
        except (IndexError, RuntimeError) as e:
            print(str(e))
            exc_type, exc_value, exc_traceback = sys.exc_info()
            print(repr(traceback.extract_tb(exc_traceback)))
            skipped += 1
            continue

        if expected == predicted:
            tp += 1
        if expected == 'N' and predicted == 'Y':
            fp += 1
        if expected == 'Y' and predicted == 'N':
            fn += 1

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    print('Precision:', precision)
    print('Recall:', precision)
    print('F1:', precision)
    print('Skipped:', skipped)

In [14]:
def get_text_up_to_question(text):
    _claim_yn = 'The evidence supports the claim:\n'
    return text[:text.find(_claim_yn) + len(_claim_yn)]

In [15]:
def get_answer_from_text(text):
    _claim_yn = 'The evidence supports the claim:\n'
    pos = text.find(_claim_yn) + len(_claim_yn)
    return text[pos]

In [16]:
def generate_answer(model, text):
    prompt = get_text_up_to_question(text)
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 1
    tokens_length = tokens.shape[1]
    if tokens_length + _length >= 1024:
        raise RuntimeError('Text is longer than 1024')
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length, 
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return get_answer_from_text(output)

In [101]:
get_answer_from_text(dev_list[0])

'Y'

In [102]:
generate_answer(model, dev_list[0])

'Y'

In [103]:
test(model, dev_list)

  1%|          | 118/19048 [00:01<04:30, 70.10it/s]

Text is longer than 1024
[<FrameSummary file <ipython-input-97-ccfa008fa64c>, line 16 in test>, <FrameSummary file <ipython-input-100-fe745795e8be>, line 7 in generate_answer>]


  2%|▏         | 424/19048 [00:05<04:14, 73.23it/s]

Text is longer than 1024
[<FrameSummary file <ipython-input-97-ccfa008fa64c>, line 16 in test>, <FrameSummary file <ipython-input-100-fe745795e8be>, line 7 in generate_answer>]


  4%|▍         | 750/19048 [00:10<04:14, 71.76it/s]

...

Text is longer than 1024
[<FrameSummary file <ipython-input-97-ccfa008fa64c>, line 16 in test>, <FrameSummary file <ipython-input-100-fe745795e8be>, line 7 in generate_answer>]


 99%|█████████▉| 18926/19048 [04:33<00:01, 74.98it/s]

Text is longer than 1024
[<FrameSummary file <ipython-input-97-ccfa008fa64c>, line 16 in test>, <FrameSummary file <ipython-input-100-fe745795e8be>, line 7 in generate_answer>]


100%|██████████| 19048/19048 [04:35<00:00, 69.26it/s]

Precision: 0.962508115126596
Recall: 0.962508115126596
F1: 0.962508115126596
Skipped: 45


Epoch 2:
Precision: 0.962508115126596
Recall: 0.962508115126596
F1: 0.962508115126596
Skipped: 45

Epoch 3:
Precision: 0.9684621220331635
Recall: 0.9684621220331635
F1: 0.9684621220331635
Skipped: 45

Epoch 4:
Precision: 0.967658052982285
Recall: 0.967658052982285
F1: 0.967658052982285
Skipped: 45


Epoch 5:
Precision: 0.9523809523809523
Recall: 0.9523809523809523
F1: 0.9523809523809523
Skipped: 18939
